In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image, ImageDraw, ImageFont
import textwrap

In [2]:
CERTIFICATE_IMAGE_TEMPLATE_PATH = "/kaggle/input/hack-certificate-template/Certificate Template (1).jpg"
SIGNATURE_PATH = "/kaggle/input/hack-certificate-template/sigs.png"
FONT_PATH = "/kaggle/input/playfair-font/PlayfairDisplay-VariableFont_wght.ttf"
TEXT_FILL = 'black'
PROTOTYPE = False

NAME_X = 60
NAME_Y = 30
NAME_FONT_SIZE = 100

PROJECT_NAME_X = 60
PROJECT_NAME_Y = 440
PROJECT_NAME_FONT_SIZE = 40
PROJECT_SPLIT_LEN = 25 # For wrapping project titles

COMPETITION_TEXT = 'Research-A-Thon\nUMKC, 2025'
COMPTETITION_X = 1000
COMPETITION_Y = 150
COMPTETITION_FONT_SIZE = 60

TRACK_X = 1000
TRACK_Y = 290
TRACK_FONT_SIZE = 60

PLACE_X = 60
PLACE_Y = 150
PLACE_FONT_SIZE = 120

SIGNATURE_LINE_WIDTH = 500

SIG_SCALING = 0.68
SIG_1_TEXT = 'Dr. Yugyung Lee, Professor,\nComputer Science'
SIG_2_TEXT = "Dr. Dianxiang Xu, Direcotor,\nComputing, Analytics, and Mathematics"
SIG_1_X = 60
SIG_1_Y = 880
SIG_2_X = 1000
SIG_2_Y = 880
SIG_1_FONT_SIZE = 30
SIG_2_FONT_SIZE = 30



In [3]:
def binarize_to_black_transparent(image_path, threshold=200):
    # Load and convert to grayscale
    image = Image.open(image_path).convert("L")

    # Create an RGBA image for output
    binarized = Image.new("RGBA", image.size)

    for y in range(image.height):
        for x in range(image.width):
            pixel = image.getpixel((x, y))
            if pixel < threshold:
                binarized.putpixel((x, y), (0, 0, 0, 255))  # Black
            else:
                binarized.putpixel((x, y), (0, 0, 0, 0))    # Transparent

    return binarized

In [4]:
def create_certificate(name, place, project_name, track):
    template_path = CERTIFICATE_IMAGE_TEMPLATE_PATH
    image = Image.open(template_path).convert("RGB")
    font_path = FONT_PATH
    signature = binarize_to_black_transparent(SIGNATURE_PATH)
    
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size

    def draw_text(text,x, y, font_size, draw=draw):
        font = ImageFont.truetype(font_path, size= font_size)
        left, bottom, right, top = draw.textbbox((0, 0), text, font=font)
        width = right-left
        height = top-bottom
        draw.text((x,y), text, font=font, fill = TEXT_FILL)
        return draw
    def draw_signature_line(draw, x_start, y, width=SIGNATURE_LINE_WIDTH, line_thickness=3, color='black'):
        x_end = x_start + width
        draw.line((x_start, y, x_end, y), fill=color, width=line_thickness)
        return draw

    draw = draw_text(name, NAME_X, NAME_Y, NAME_FONT_SIZE, draw)
    draw = draw_text(project_name, PROJECT_NAME_X, PROJECT_NAME_Y, PROJECT_NAME_FONT_SIZE, draw)
    draw = draw_text(COMPETITION_TEXT, COMPTETITION_X, COMPETITION_Y, COMPTETITION_FONT_SIZE, draw)
    draw = draw_text(track, TRACK_X, TRACK_Y, TRACK_FONT_SIZE, draw)
    draw = draw_text(place, PLACE_X, PLACE_Y, PLACE_FONT_SIZE, draw)
    draw = draw_text(SIG_1_TEXT, SIG_1_X, SIG_1_Y, SIG_1_FONT_SIZE, draw)
    # draw = draw_signature_line(draw, SIG_1_X, SIG_1_Y)
    draw = draw_text(SIG_2_TEXT, SIG_2_X, SIG_2_Y, SIG_2_FONT_SIZE, draw)
    # Resize signature if needed
    signature = signature.resize((int(SIG_SCALING*signature.width), int(SIG_SCALING*signature.height)))  # Width, height in pixels
    
    # Set position (bottom-right corner with some padding)
    sig_x = SIG_1_X
    sig_y = SIG_1_Y-80
    
    # Paste using alpha channel if available
    image.paste(signature, (sig_x, sig_y), signature)    
    return image

In [5]:
pth = "/kaggle/input/hackathon-certificates/UMKC_2025_Spring_Research-A-Thon_ Winners.csv"
cols = ['prize', 'place', 'project_name', 'names', 'email', 'poster_path', 'video_path']
winners = pd.read_csv(pth).reset_index(drop=True)
winners.columns = cols
winners

,prize,place,project_name,names,email,poster_path,video_path
0,Prizes,Category,Project Title,Name,email,Poster,Video
1,NaN,AI/DS PhD Track,NaN,NaN,NaN,NaN,NaN
2,"$1,000.00",1st,Team 3. SAM-KG: Knowledge Graph-Enhanced Tumor...,Saeed Alqarni,saacfb@umkc.edu,https://drive.google.com/open?id=1OtLqxzUsH2OS...,https://drive.google.com/file/d/1lVPczqeZKk8JB...
3,$700.00,2nd,Team 1. NavClone: Imitation Learning Enhanced ...,Ahmed Alanazi,aha85b@umkc.edu,https://drive.google.com/open?id=1X6Fjl4dpLQ8B...,https://umkc.box.com/s/jmr7grkcp02axqtwbscbk6t...
4,$500.00,3rd,Team 2. KARMA: A Knowledge Graph Framework for...,Udiptaman Das,ud3d4@umkc.edu,https://drive.google.com/open?id=1u8NKwpjzxijt...,https://drive.google.com/file/d/1SAzC-dnqE7wFG...
5,$100.00,HM,Team 4. PostCorrect: Real-Time Posture Correct...,"Abhinav Kochar, Ahmed Alanazi","arkrnr@umkc.edu, aha85b@umkc.edu",https://drive.google.com/open?id=1TJHaVblDVKwR...,https://drive.google.com/drive/folders/1Pq1Mii...
6,$100.00,HM,Team 6. G2SEM: Gradient-Guided Semantic Relati...,Muhammad Imran,mi3dr@umkc.edu,https://drive.google.com/open?id=1vINDj2FrHGPJ...,https://drive.google.com/file/d/1pYGtzAPlAbfR4...
7,$100.00,HM,Team 7. Batching Model Slices for Resource-eff...,Waleed Mubark,wmd8c@umkc.edu,https://drive.google.com/open?id=1o_7R-2E__ihd...,https://youtu.be/EccwZ26RPm8
8,NaN,AI/DS MS Track,NaN,NaN,NaN,NaN,NaN
9,"$1,000.00",1st,Team 17. Towards Smarter Cyber Defense: Anomal...,"Prathyu Adari, Riley Bruce, Ransom Ward, Katy ...","prathyuadari@umkc.edu, rgbmrb@umkc.edu, rdw2md...",https://drive.google.com/open?id=1aPioNuEeYK32...,https://www.youtube.com/watch?v=urFNDvC0exM


In [6]:
col_list = ['place', 'project_name', 'names']
phd_track_winners = winners[2:8][col_list]
ms_track_winners = winners[9:15][col_list]
ug_track_winners = winners[16:17][col_list]
q_track_winners = winners[18:21][col_list]
winner_dict = {"AI and Data Science, PhD Track": phd_track_winners,
              "AI and Data Science, MS Track": ms_track_winners,
              "AI and Data Science, Undergraduate Track": ug_track_winners,
              "AI and Data Science, Quantum Track": q_track_winners}
phd_track_winners.head()

,place,project_name,names
2,1st,Team 3. SAM-KG: Knowledge Graph-Enhanced Tumor...,Saeed Alqarni
3,2nd,Team 1. NavClone: Imitation Learning Enhanced ...,Ahmed Alanazi
4,3rd,Team 2. KARMA: A Knowledge Graph Framework for...,Udiptaman Das
5,HM,Team 4. PostCorrect: Real-Time Posture Correct...,"Abhinav Kochar, Ahmed Alanazi"
6,HM,Team 6. G2SEM: Gradient-Guided Semantic Relati...,Muhammad Imran


In [7]:
split_len = PROJECT_SPLIT_LEN
join_char = '_'
for track, df in winner_dict.items():
    for _, row in df.iterrows():
        place_str = str(row['place'])
        if place_str != "HM":
            place = place_str + ' Place'
        else:
            place = "Honorable\nMention"

        # Extract project name and wrap it
        raw_project_name = str(row['project_name']).split('. ')[1] # Delete team numbers from project names
        wrapped_lines = textwrap.wrap(raw_project_name, width=split_len)
        new_project_name = '\n'.join(wrapped_lines)

        # Handle multiple names
        name_list = row['names'].split(', ')
        for name in [nm.strip() for nm in name_list]:
            underscore_name = join_char.join(name.split(" ")) #names to avoid spaces in filenames
            underscore_track = join_char.join(track.split(", ")[1].split(" ")) #tracks to avoid spaces in filenames
            file_name = f'/kaggle/working/{underscore_name}_{underscore_track}.jpg'
            split_track = '\n'.join(track.split(', '))
            image = create_certificate(name, place, new_project_name, split_track)
            image.save(file_name)
            if PROTOTYPE:
                break
        if PROTOTYPE:
            break
    if PROTOTYPE:
        break
